In [63]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.metrics import r2_score

In [92]:
pd.set_option('display.max_rows', None)

In [5]:
df_read = pd.read_csv('postings.csv')

In [7]:
df_read.head(5)

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0


In [9]:
df = df_read[['company_name','title','description','min_salary','max_salary','pay_period','formatted_work_type','currency','normalized_salary','formatted_experience_level']]

In [11]:
df = df[df.formatted_work_type == 'Full-time']

In [13]:
df = df[df.currency == 'USD']

In [15]:
df = df.dropna(subset=['company_name'])
#df = df.dropna(subset=['zip_code'])

In [17]:
df = df.dropna(subset=['normalized_salary'])
df = df[df['normalized_salary'] > 15000]

In [19]:
# fix low salaries
df['annual_salary'] = np.where((df.normalized_salary < 30000) & (df.formatted_experience_level == 'Mid-Senior level') & (df.pay_period == 'YEARLY'),
    df.normalized_salary * 12, df.normalized_salary)

In [21]:
# fix high salaries
df['annual_salary'] = np.where((df.max_salary > 50000) & (df.normalized_salary > df.max_salary * 2), df.max_salary, df.normalized_salary)
df = df[df['annual_salary'] < 10000000]

In [25]:
#df.sort_values(by='normalized_salary', ascending=False).head(100)
df.sort_values(by='annual_salary', ascending=False).head(10)

,company_name,title,description,min_salary,max_salary,pay_period,formatted_work_type,currency,normalized_salary,formatted_experience_level,annual_salary
116367,Goliath Partners,Quantitative Researcher,Goliath Partners is working with a global prop...,700000.0,1200000.0,YEARLY,Full-time,USD,950000.0,Mid-Senior level,950000.0
4781,"Platinum Legal Search Group, LLC",Partner (& Groups w/ Portable Business for Top...,We're recruiting for a Top Law Firm seeking Pa...,350000.0,1500000.0,YEARLY,Full-time,USD,925000.0,NaN,925000.0
4735,"Platinum Legal Search Group, LLC",Partner (& Groups w/ Portable Business for Top...,We're recruiting for a Top Law Firm seeking Pa...,350000.0,1500000.0,YEARLY,Full-time,USD,925000.0,NaN,925000.0
91967,"Platinum Legal Search Group, LLC","Corporate, Litigation, Intellectual Property ,...",We're recruiting for a Top Law Firm seeking Pa...,350000.0,1500000.0,YEARLY,Full-time,USD,925000.0,NaN,925000.0
4908,"Platinum Legal Search Group, LLC",Partner (& Groups w/ Portable Business for Top...,We're recruiting for a Top Law Firm seeking Pa...,350000.0,1500000.0,YEARLY,Full-time,USD,925000.0,NaN,925000.0
113069,Goldman Lloyds,Head of Front Office Technology,Job Title: Head of Front Office Technology\nCo...,750000.0,1000000.0,YEARLY,Full-time,USD,875000.0,NaN,875000.0
117057,Goliath Partners,Quantitative Researcher,Goliath has partnered with a well concealed ye...,650000.0,1000000.0,YEARLY,Full-time,USD,825000.0,Mid-Senior level,825000.0
115087,Radley James,Network Engineer,"**PAYING UP TO $750,000 TOTAL COMPENSATION****...",750000.0,750000.0,YEARLY,Full-time,USD,750000.0,Mid-Senior level,750000.0
54411,Emburse,Senior Sales Account Executive - Enterprise,At Emburse our mission is to help make our use...,NaN,NaN,YEARLY,Full-time,USD,750000.0,Mid-Senior level,750000.0
115360,Radley James,Python Developer,**Offering up to $650K and a comprehensive rel...,750000.0,750000.0,YEARLY,Full-time,USD,750000.0,Mid-Senior level,750000.0


In [27]:
df.shape

(28471, 11)

In [29]:
df = pd.get_dummies(df, columns=['pay_period'])

In [342]:
#df['log_salary'] = np.log1p(df['annual_salary'])

In [31]:
pay_period_features = df[['pay_period_BIWEEKLY', 'pay_period_HOURLY',
                          'pay_period_MONTHLY', 'pay_period_WEEKLY', 'pay_period_YEARLY']]
y = df['annual_salary'].values

In [350]:
#df['text'] = df['company_name'] + ' ' + df['title']
#df['text'] = df['text'].astype(str)

In [33]:
df["text"] = df["title"].fillna("") 
df["text"] = df["title"].astype(str)

In [35]:
text_data = df['text']

In [37]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [39]:
def get_bert_embeddings(texts, tokenizer, model, batch_size=32):
    model.eval()
    embeddings = []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            inputs = tokenizer(batch.tolist(), return_tensors="pt", padding=True, truncation=True, max_length=20)
            outputs = model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token
            embeddings.append(cls_embeddings.cpu().numpy())
    return np.vstack(embeddings)

In [41]:
bert_embeddings = get_bert_embeddings(text_data, tokenizer, bert_model)

In [360]:
#X = bert_embeddings

In [49]:
X = np.hstack((bert_embeddings, pay_period_features.values))

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

In [55]:
xgb_model = xgb.XGBRegressor(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

In [57]:
model = xgb.XGBRegressor()
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [59]:
y_pred = model.predict(X_test)
#y_pred = xgb_model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"XGBoost RMSE: {rmse:,.0f}")

XGBoost RMSE: 46,795


In [65]:
r2_score(y_test, y_pred)

0.4746700628785828

In [73]:
job_titles = df['title'].tolist()

In [75]:
job_title_to_embedding = {job: emb for job, emb in zip(job_titles, bert_embeddings)}

# Save to file
import joblib
joblib.dump(job_title_to_embedding, "bert_job_embeddings.pkl")

['bert_job_embeddings.pkl']

In [77]:
joblib.dump(xgb_model, 'xgb_salary_model.pkl')

['xgb_salary_model.pkl']